## Job Density
### Los Angeles County

In the Week 4 Individual Coding Assignment, I identified neighborhoods with the highest population densities in LA County. 

Now I'll build off of what I worked on in Week 3 to calculate job density from the total job count. I will first call in job count data from OntheMap, and copy the work I did in Week 3.

In [1]:
# for spatial data
import geopandas as gpd
# for general data wrangling tasks
import pandas as pd

# for plotting
import matplotlib.pyplot as plt
# for basemaps
import contextily as ctx

# for network analysis
import networkx as nx
# for street network analysis
import osmnx as ox

In [2]:
LA_jobs = gpd.read_file('AYJ-GroupData/points_2018.shp')

In [3]:
keep_job_columns = ['id',
 'c000',
 'geometry']

In [4]:
LA_jobs = LA_jobs[keep_job_columns]

In [5]:
LA_jobs.columns = ['GEOID', 'Job Count', 'geometry']

Because this is point data, I need to merge the areas of each Census tract by their GEOID from the .csv file I produced from the population density analysis. 

In [6]:
LA_jobs.head()

,GEOID,Job Count,geometry
0,060374600003006,12.0,POINT (-2001134.000 -138945.141)
1,060374300025013,3.0,POINT (-1991025.125 -145032.016)
2,060374300025015,1.0,POINT (-1991167.125 -145140.297)
3,060374300025014,21.0,POINT (-1991184.625 -145096.594)
4,060374300025021,4.0,POINT (-1991041.750 -145197.812)


In [7]:
LA_pop = gpd.read_file('AYJ-GroupData/cleanedLApop_areas.csv')

In [8]:
LA_pop

,field_1,GEOID,area,Pop Density,geometry
0,0,14000US06037101110,1190263.184656883,0.003598363836847277,None
1,1,14000US06037101122,2754669.980421015,0.0012360827337580347,None
2,2,14000US06037101210,677981.8833714458,0.009361607080764237,None
3,3,14000US06037101220,728204.6899016033,0.005083735454244634,None
4,4,14000US06037101300,2689564.1879190835,0.0014441001324474996,None
...,...,...,...,...,...
2341,2341,14000US06037980031,27111981.338929184,4.33387728219205e-05,None
2342,2342,14000US06037980033,30012880.890518565,5.33104438003304e-07,None
2343,2343,14000US06037990100,180339344.96581003,0.0,None
2344,2344,14000US06037990200,216681423.73547158,0.0,None


In [9]:
LA_pop.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   field_1      2346 non-null   object  
 1   GEOID        2346 non-null   object  
 2   area         2346 non-null   object  
 3   Pop Density  2346 non-null   object  
 4   geometry     0 non-null      geometry
dtypes: geometry(1), object(4)
memory usage: 91.8+ KB


Sigh, I need to delete the "14000US" from the front of each LA_pop GEOID. 

Using .replace()
Source: https://stackoverflow.com/questions/37372603/how-to-remove-specific-substrings-from-a-set-of-strings-in-python 

Using .strip()
Source: https://www.programiz.com/python-programming/methods/string/strip

Using .removeprefix()
Source: https://stackoverflow.com/questions/1038824/how-do-i-remove-a-substring-from-the-end-of-a-string-in-python 
url = 'abcdc.com'
url.removesuffix('.com')    # Returns 'abcdc'
url.removeprefix('abcdc.')  # Returns 'com'

In [10]:
LA_pop.GEOID = [x.replace('14000US', '') for x in LA_pop.GEOID] 
#LA_pop.GEOID = [x.strip('14000US') for x in LA_pop.GEOID]     Source: https://towardsdatascience.com/5-methods-to-remove-the-from-your-data-in-python-and-the-fastest-one-281489382455 
# didn't work: cleanedFIPS = LA_pop['GEOID'].strip('14000US') 
## .removeprefix() didn't work either - do we not have Python 3.9 or newer? 

In [11]:
LA_pop.head()

,field_1,GEOID,area,Pop Density,geometry
0,0,06037101110,1190263.184656883,0.003598363836847277,None
1,1,06037101122,2754669.980421015,0.0012360827337580347,None
2,2,06037101210,677981.8833714458,0.009361607080764237,None
3,3,06037101220,728204.6899016033,0.005083735454244634,None
4,4,06037101300,2689564.1879190835,0.0014441001324474996,None


In [12]:
LA_jobs = LA_jobs.merge(LA_pop, on='GEOID')# Merge with `merge` method on shared variable 

In [13]:
LA_jobs

,GEOID,Job Count,geometry_x,field_1,area,Pop Density,geometry_y


I have no idea why this isn't working ;_; 
